## 2. Sentiment Analysis
In this exercise, we will classify the sentiment of text documents. Complete the code with TODO tag.

References and Further Readings:
+ http://www.nltk.org/howto/sentiment.html
+ https://www.nltk.org/api/nltk.sentiment.html
+ http://datameetsmedia.com/vader-sentiment-analysis-explained/
+ https://github.com/cjhutto/vaderSentiment
+ https://marcobonzanini.com/2015/05/17/mining-twitter-data-with-python-part-6-sentiment-analysis-basics/
+ https://github.com/marrrcin/ml-twitter-sentiment-analysis


### 2.1. Classification approach

Classification approach looks at previously labeled data in order to determine the sentiment of never-before-seen sentences. It involves training a model using previously seen text to predict/classify the sentiment of some new input text. The nice thing is that, with a greater volume of data, we generally get better prediction or classification results. However, unlike the lexical approach, we need previously labeled data.

In [2]:
import nltk
from nltk.classify import NaiveBayesClassifier
nltk.download('vader_lexicon')
nltk.download('movie_reviews')
from nltk.corpus import subjectivity
from nltk.corpus import movie_reviews
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

# n_instances = 100
# subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
# obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
# len(subj_docs), len(obj_docs)

n_instances = None
if n_instances is not None:
    n_instances = int(n_instances/2)

pos_docs = [(list(movie_reviews.words(pos_id)), 'pos') for pos_id in movie_reviews.fileids('pos')[:n_instances]]
neg_docs = [(list(movie_reviews.words(neg_id)), 'neg') for neg_id in movie_reviews.fileids('neg')[:n_instances]]
len(pos_docs), len(neg_docs)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Connor\AppData\Roaming\nltk_data...
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Connor\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


(1000, 1000)

Each document is represented by a tuple (sentence, label). The sentence is tokenized, so it is represented by a list of strings:

In [3]:
pos_docs[0]

(['films',
  'adapted',
  'from',
  'comic',
  'books',
  'have',
  'had',
  'plenty',
  'of',
  'success',
  ',',
  'whether',
  'they',
  "'",
  're',
  'about',
  'superheroes',
  '(',
  'batman',
  ',',
  'superman',
  ',',
  'spawn',
  ')',
  ',',
  'or',
  'geared',
  'toward',
  'kids',
  '(',
  'casper',
  ')',
  'or',
  'the',
  'arthouse',
  'crowd',
  '(',
  'ghost',
  'world',
  ')',
  ',',
  'but',
  'there',
  "'",
  's',
  'never',
  'really',
  'been',
  'a',
  'comic',
  'book',
  'like',
  'from',
  'hell',
  'before',
  '.',
  'for',
  'starters',
  ',',
  'it',
  'was',
  'created',
  'by',
  'alan',
  'moore',
  '(',
  'and',
  'eddie',
  'campbell',
  ')',
  ',',
  'who',
  'brought',
  'the',
  'medium',
  'to',
  'a',
  'whole',
  'new',
  'level',
  'in',
  'the',
  'mid',
  "'",
  '80s',
  'with',
  'a',
  '12',
  '-',
  'part',
  'series',
  'called',
  'the',
  'watchmen',
  '.',
  'to',
  'say',
  'moore',
  'and',
  'campbell',
  'thoroughly',
  'researche

We separately split subjective and objective instances to keep a balanced uniform class distribution in both train and test sets.

In [12]:
# COMPLETE: split training and testing data as 80/20
# from sklearn.model_selection import train_test_split
# train_pos_docs, test_pos_docs = train_test_split(pos_docs, test_size=0.2)
# train_neg_docs, test_neg_docs = train_test_split(neg_docs, test_size=0.2)
train_pos_docs, test_pos_docs = pos_docs[:80], pos_docs[80:100]
train_neg_docs, test_neg_docs = neg_docs[:80], neg_docs[80:100]

training_docs = train_pos_docs+train_neg_docs
testing_docs = test_pos_docs+test_neg_docs
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

all_words_neg[:5]

['films', 'adapted', 'from', 'comic', 'books']

We use simple unigram word features, handling negation:

In [13]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
print(len(unigram_feats))
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

2721


We apply features to obtain a feature-value representation of our datasets:

In [14]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)
print(training_set[0])

({'contains(,)': True, 'contains(the)': True, 'contains(.)': True, 'contains(a)': True, 'contains(and)': True, 'contains(of)': True, "contains(')": True, 'contains(to)': True, 'contains(is)': True, 'contains(in)': True, 'contains(s)': True, 'contains(")': True, 'contains(-)': True, 'contains(that)': True, 'contains(it)': True, 'contains(()': True, 'contains())': True, 'contains(as)': True, 'contains(with)': True, 'contains(his)': False, 'contains(he)': True, 'contains(for)': True, 'contains(this)': True, 'contains(film)': True, 'contains(i)': True, 'contains(but)': True, 'contains(on)': True, 'contains(t)': True, 'contains(are)': True, 'contains(an)': True, 'contains(by)': True, 'contains(the_NEG)': False, 'contains(movie)': False, 'contains(you)': True, 'contains(who)': True, 'contains(be)': True, 'contains(,_NEG)': False, 'contains(not)': True, 'contains(one)': True, 'contains(from)': True, 'contains(at)': True, 'contains(has)': True, 'contains(was)': True, 'contains(?)': True, 'cont

We can now train our classifier on the training set, and subsequently output the evaluation results:

In [15]:
# TODO: Use Naive Bayes to train the sentiment classifier
naive_bayes = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(naive_bayes, training_set)
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.725
F-measure [neg]: 0.717948717948718
F-measure [pos]: 0.7317073170731707
Precision [neg]: 0.7368421052631579
Precision [pos]: 0.7142857142857143
Recall [neg]: 0.7
Recall [pos]: 0.75


### 2.2. Lexical approach

Lexical approaches aim to map words to sentiment by building a lexicon or a 'dictionary of sentiment'. We can use this dictionary to assess the sentiment of phrases and sentences, without the need of looking at anything else. Sentiment can be categorical – such as {negative, neutral, positive} – or it can be numerical – like a range of intensities or scores. Lexical approaches look at the sentiment category or score of each word in the sentence and decide what the sentiment category or score of the whole sentence is. The power of lexical approaches lies in the fact that we do not need to train a model using labeled data, since we have everything we need to assess the sentiment of sentences in the dictionary of emotions. VADER is an example of a lexical method.

In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Run the lexical approach

In [17]:
sid = SentimentIntensityAnalyzer()
for doc in testing_docs:
    doc = " ".join(doc[0])
    print(doc[:100] + "...")
    ss = sid.polarity_scores(doc)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()

just how inseparable is the team of sgt . martin riggs ( mel gibson ) and sgt . roger murtaugh ( dan...
compound: 0.7656, neg: 0.103, neu: 0.791, pos: 0.106, 
since 1990 , the dramatic picture has undergone a certain change of style . now , instead of emphasi...
compound: 0.9982, neg: 0.027, neu: 0.79, pos: 0.183, 
a big surprise to me . the good trailer had hinted that they pulled the impossible off , but making ...
compound: 0.9779, neg: 0.145, neu: 0.65, pos: 0.205, 
after having heard so many critics describe " return to me " as an old - fashioned hollywood romance...
compound: 0.9995, neg: 0.042, neu: 0.738, pos: 0.22, 
wild things is a suspenseful thriller starring matt dillon , denise richards , and neve campbell tha...
compound: 0.9965, neg: 0.07, neu: 0.77, pos: 0.16, 
* * * * * * minor plot spoilers in review * * * * * * * * * * * * no major spoilers are in review * ...
compound: 0.9904, neg: 0.073, neu: 0.82, pos: 0.107, 
are you tired of all the hot new releases being gone 

### 2.3 Comparing two approaches

First we can transform the sentiment score by the lexical approach into label by the following rules:

+ positive sentiment: compound score > 0
+ negative sentiment: compound score <= 0

In [21]:
def lexical_sentiment(doc, sid=None):
    """COMPLETE: return the label 'pos' or 'neg' for a document"""
    if sid is None: sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(doc)
    if ss["neg"] > ss["pos"]:
        return "neg"
    else:
        return "pos"

for doc in testing_docs:
    doc = " ".join(doc[0])
    label = lexical_sentiment(doc, sid)
    print(doc[:100] + "...", label)

just how inseparable is the team of sgt . martin riggs ( mel gibson ) and sgt . roger murtaugh ( dan... pos
since 1990 , the dramatic picture has undergone a certain change of style . now , instead of emphasi... pos
a big surprise to me . the good trailer had hinted that they pulled the impossible off , but making ... pos
after having heard so many critics describe " return to me " as an old - fashioned hollywood romance... pos
wild things is a suspenseful thriller starring matt dillon , denise richards , and neve campbell tha... pos
* * * * * * minor plot spoilers in review * * * * * * * * * * * * no major spoilers are in review * ... pos
are you tired of all the hot new releases being gone by the time you get to the video store ? waffle... neg
many people dislike french films for their lack of closure . while possibly shallow , i ' ve often h... pos
the keen wisdom of an elderly bank robber , the naive ambitions of a sexy hospital nurse , and a par... pos
robert benton has assembled 

Now we evaluate the lexical approach by computing accuracy metrics

In [73]:
from collections import defaultdict
from nltk.metrics import (accuracy as eval_accuracy, precision as eval_precision,
        recall as eval_recall, f_measure as eval_f_measure)

gold_results = defaultdict(set)
test_results = defaultdict(set)
acc_gold_results = []
acc_test_results = []
labels = set()
num = 0
for i, (text, label) in enumerate(testing_docs):
    labels.add(label)
    gold_results[label].add(i)
    acc_gold_results.append(label)
    observed = lexical_sentiment(" ".join(text), sid)
    num += 1
    acc_test_results.append(observed)
    test_results[observed].add(i)
metrics_results = {}

# COMPLETE: compute the accuracy metrics
metrics_results["accuracy"] = eval_accuracy(acc_gold_results, acc_test_results)
# metrics_results["precision"] = eval_precision(gold_results["pos"], test_results["pos"])
# metrics_results["recall"] = eval_recall(gold_results["pos"], test_results["pos"])
# metrics_results["f_measure"] = eval_f_measure(gold_results["pos"],test_results["pos"])
    
TP = len(test_results["pos"].intersection(gold_results["pos"]))
FP = len(test_results["pos"].intersection(gold_results["neg"]))
FN = len(test_results["neg"].intersection(gold_results["pos"]))

metrics_results["precision"] = TP/(TP+FP)
metrics_results["recall"] = TP/(TP+FN)
metrics_results["f_measure"] = 2*metrics_results["precision"]*metrics_results["recall"]/(metrics_results["precision"]+metrics_results["recall"])

for result in sorted(metrics_results):
        print('{0}: {1}'.format(result, metrics_results[result]))

accuracy: 0.575
f_measure: 0.6222222222222222
precision: 0.56
recall: 0.7
